# Double Q-Learner

Double Q-Learning was created by Hado van Hasselt in 2010. The idea is that because a normal q-learner uses *max* that it overestimates the action values. To help solve this Hasselt introduces a second Q table that we will use to figure out the max value action estimate.

Here is the abstract from the paper:  
In some stochastic environments the well-known reinforcement learning algorithm
Q-learning performs very poorly. This poor performance is caused by large
overestimations of action values. These overestimations result from a positive
bias that is introduced because Q-learning uses the maximum action value as an
approximation for the maximum expected action value. We introduce an alternative
way to approximate the maximum expected value for any set of random
variables. The obtained double estimator method is shown to sometimes underestimate
rather than overestimate the maximum expected value. We apply the
double estimator to Q-learning to construct Double Q-learning, a new off-policy
reinforcement learning algorithm. We show the new algorithm converges to the
optimal policy and that it performs well in some settings in which Q-learning performs
poorly due to its overestimation

**Equations**   
Remember, this is the Q-learning update formula  
$$Q'(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha * \big( r_t + \gamma * max_a Q(s_{t+1},a) - Q(s_t,a_t) \big) $$  

When using a Double Q-learner you will have 2 update equations that you randomly select which one you update. 
Choose UPDATE(A) or UPDATE(B) randomly  
UPDATE(A)
$$a^* = arg \, max_a Q^A(s',a)$$
$$Q^A(s,a) \leftarrow Q^A(s,a) + \alpha(s,a) * \big( r + \gamma * Q^B(s',a^*) - Q^A(s,a) \big) $$
UPDATE(B)
$$a^* = arg \, max_a Q^B(s',a)$$
$$Q^B(s,a) \leftarrow Q^B(s,a) + \alpha(s,a) * \big( r + \gamma * Q^A(s',a^*) - Q^B(s,a) \big) $$  

Now, to put this in simplier terms, if you are doing UPDATE(A) you will do the equation like normal BUT when you find the future rewards ($Q^B(s',a^*)$) you are finding the **MAX** value from the $Q^A$ table and taking that index and getting the actual value from $Q^B$.  

In an attempt to cement this in your brains here is an example:  
$Q^A$ has this entry in state X: [ 0.1, 0.2, 0.12, 0.005]  
$Q^B$ has this entry in state X: [ 0.9, 0.5, 0.30, 0.020]  

What is the expected future value if you are doing UPDATE(A)?

In [3]:
from doubleq import DoubleQQuestion1
DoubleQQuestion1(0.0)

Incorrect.
  First, you need to find the index of MAX(Q^A)
  Then, you find the expected value at that index in Q^B


I am going to do one more question before we get into the implementation of a full Double Q-Leaner.  

Code up both UpdateA and UpdateB and then run the test below.

In [4]:
import numpy as np

In [11]:
def UpdateA(QA, QB, state, action, nstate, alpha=1, reward=1, gamma=1, done=False):
    #Find a* (Index of the max value in s') in QA
    a_star_idx = 0 #TODO
    #Find QB[s',a*]
    future_value = 0 #TODO
    QA[state][action] += alpha * (reward + gamma * future_value * (not done) - QA[state][action])

In [12]:
def UpdateB(QA, QB, state, action, nstate, alpha=1, reward=1, gamma=1, done=False):
    #Find a* (Index of the max value in s') in QA
    a_star_idx = 0 #TODO
    #Find QA[s',a*]
    future_value = 0 #TODO
    QB[state][action] += alpha * (reward + gamma * future_value * (not done) - QB[state][action])

In [13]:
nS = 10 #10 States
nA = 4 #4 Actions
QA = np.zeros((nS,nA), dtype=np.float)
QB = np.zeros((nS,nA), dtype=np.float)

#Set the maximum expected value for each of the 4 actions for state 1
QA[1] = (1,2,3,4)
QB[1] = (8,7,6,5)

QA[2] = (5,6,2,1)
QB[2] = (7,5,6,4)

#Starting at state 1, take action 3, and end up in state 2
UpdateA(QA,QB, 1, 3, 2)
if QA[1,3] == 6.0:
    print('Success')
else:
    print('Fail. Try again.')
#Repeat with QB being randomly selected
UpdateB(QA,QB, 1, 3, 2)
if QB[1,3] == 6.0:
    print('Success')
else:
    print('Fail. Try again.')

Fail. Try again.
Fail. Try again.


If you were having troubles here is how I would explain what was going on. I am going to use UpdateA as an example as it should be easy enough for you to switch $Q^A$ and $Q^B$.  

To find $$a^* = arg \, max_a Q^A(s',a)$$ you need to find the index of the max value in the next state (nstate in code) of  $Q^A$. Numpy has np.argmax for just this reason.  
CODE: a_star_idx = np.argmax(QB[nstate])

To find $$Q^B(s',a^*)$$ you would take the index from the previous answer and put it against the next state (nstate) of $Q^B$.  
CODE: future_value = QA[nstate,a_star_idx]

*References*  
Hasselt, H. V. (2010). Double Q-learning. Advances in Neural Information Processing Systems 23,2613-2621. Retrieved from http://papers.nips.cc/paper/3964-double-q-learning.pdf